In [20]:
import threading
import time
import os 
import pandas as pd
import datetime

In [21]:
aggregate_report = pd.read_csv(".//aggr_report.csv")
aggregate = pd.read_csv(".//aggregate.csv")

In [22]:
aggregate_report.head()

,timeStamp,elapsed,label,responseCode,responseMessage,threadName,dataType,success,failureMessage,bytes,sentBytes,grpThreads,allThreads,URL,Latency,IdleTime,Connect
0,1666260780795,46,12:13:00,200,OK,Thread Group 1-1,NaN,True,NaN,5243147,119,2,2,http://172.16.210.132/1-6,1,0,0
1,1666260780830,34,12:13:00,200,OK,Thread Group 1-2,NaN,True,NaN,5243147,119,3,3,http://172.16.210.132/1-6,5,0,1
2,1666260780861,22,12:13:00,200,OK,Thread Group 1-3,NaN,True,NaN,5243147,119,3,3,http://172.16.210.132/1-6,2,0,1
3,1666260780898,18,12:13:00,200,OK,Thread Group 1-4,NaN,True,NaN,5243147,119,4,4,http://172.16.210.132/1-6,2,0,2
4,1666260780928,21,12:13:00,200,OK,Thread Group 1-5,NaN,True,NaN,5243147,119,5,5,http://172.16.210.132/1-6,2,0,2


In [23]:
aggregate["Error %"] = aggregate["Error %"].str.replace("%","")
aggregate["Error %"] = aggregate["Error %"].str.replace(",",".")
aggregate["Error %"] = pd.to_numeric(aggregate["Error %"], downcast="float")
aggregate["Error %"] = aggregate["# Samples"] * aggregate["Error %"] /100
aggr_1 = aggregate[['Label','Throughput','Error %']].copy()
aggr_1.rename(columns = {'Label':'label','Error %' : 'errors'}, inplace = True)

In [24]:
aggregate_report = aggregate_report.groupby(['label'],axis=0,as_index=False).mean()
aggr_2 = aggregate_report[['label','elapsed','Latency']].copy()

In [25]:
aggr_df = pd.merge(aggr_1, aggr_2, on='label')
aggr_df.head()

,label,Throughput,errors,elapsed,Latency
0,12:13:00,"31,11111",0.0,26.571429,2.571429
1,12:13:01,"6,14979",0.0,1414.928571,1363.892857
2,12:13:02,"5,50206",0.0,39.250000,1.500000
3,12:13:03,"6,75676",0.0,44.200000,1.800000
4,12:13:04,"32,25806",0.0,31.000000,1.000000


In [26]:
if not os.path.isfile('client.csv'):
   aggr_df.to_csv('client.csv',index=False)
else: 
   aggr_df.to_csv('client.csv', index = False,mode='a', header=False)